## Data transformation

This is the second automatically graded exercise for JODA. The objective here is to get our hands dirty with data. 

The context of this particular analysis is a fictional company that routinely runs different machine learning operations. 

We have generated a dataset that has the following columns or properties (to be engineered into features):

* Date
* Department
* ML Task ID
* ML Method
* Task Category
* Model Complexity (Parameters)
* Training Data Size (GB)
* Training Duration (Hours)
* Hardware Used
* Energy Consumption (kWh)
* CO2 Emissions (Kg)
* Cloud Provider

Moreover, there is a secondary dataset that includes information about the energy sources for the different cloud providers:

* Cloud Provider    
* Green Energy


Install the required packages using requirements.txt


In [40]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Import the needed packages

In [41]:
import pandas as pd

Read the two data files into data frames and merge them into one

In [42]:
df_co2 = pd.read_excel('data/cloud-providers.xlsx', engine='openpyxl')
df_providers = pd.read_excel('data/co2-emissions.xlsx', engine='openpyxl')
df = pd.merge(df_co2, df_providers, on='Cloud Provider')
df.head()

,Cloud Provider,Green Energy,Date,Department,ML Task ID,ML Method,Task Category,Model Complexity (Parameters),Training Data Size (GB),Training Duration (Hours),Hardware Used,Energy Consumption (kWh),CO2 Emissions (Kg)
0,CloudTech,Unknown,2024-02-01,Operations,Task_2,Linear Regression,Regression,1.053744e+03,12.288902,0.994852,TPU,12.260144,4.904058
1,CloudTech,Unknown,2024-02-01,R&D,Task_3,Decision Tree,Machine Translation,1.098484e+03,127.561567,1.903623,CPU,27.869589,11.147836
2,CloudTech,Unknown,2024-02-03,Customer Support,Task_10,RNN,Text Generation,2.188188e+06,16.194136,11.772296,TPU,195.540627,78.216251
3,CloudTech,Unknown,2024-02-05,Finance,Task_17,RNN,Clustering,2.112308e+06,3.722296,12.892916,GPU,187.786419,75.114568
4,CloudTech,Unknown,2024-02-05,Marketing,Task_19,Linear Regression,Image Classification,8.349115e+02,74.796534,0.974523,CPU,18.340844,7.336338


Aggregate the data to department level and calculate the total of CO2 emissions for each department.
Rename the emissions column to 'co2_emissions_kg'

In [43]:
total_co2 = df.groupby('Department')['CO2 Emissions (Kg)'].sum().rename('co2_emissions_kg')
total_co2.head()

Department
Customer Support    12565.569898
Finance             13568.637182
Human Resources     15256.236043
Marketing           12821.756125
Operations          15004.901708
Name: co2_emissions_kg, dtype: float64

Rename CO2 emission column to 'co2_emissions_kg'

In [44]:
df = df.rename(columns={'CO2 Emissions (Kg)': 'co2_emissions_kg'})
df.head()

,Cloud Provider,Green Energy,Date,Department,ML Task ID,ML Method,Task Category,Model Complexity (Parameters),Training Data Size (GB),Training Duration (Hours),Hardware Used,Energy Consumption (kWh),co2_emissions_kg
0,CloudTech,Unknown,2024-02-01,Operations,Task_2,Linear Regression,Regression,1.053744e+03,12.288902,0.994852,TPU,12.260144,4.904058
1,CloudTech,Unknown,2024-02-01,R&D,Task_3,Decision Tree,Machine Translation,1.098484e+03,127.561567,1.903623,CPU,27.869589,11.147836
2,CloudTech,Unknown,2024-02-03,Customer Support,Task_10,RNN,Text Generation,2.188188e+06,16.194136,11.772296,TPU,195.540627,78.216251
3,CloudTech,Unknown,2024-02-05,Finance,Task_17,RNN,Clustering,2.112308e+06,3.722296,12.892916,GPU,187.786419,75.114568
4,CloudTech,Unknown,2024-02-05,Marketing,Task_19,Linear Regression,Image Classification,8.349115e+02,74.796534,0.974523,CPU,18.340844,7.336338


Create a function that picks the most common value among in a Pandas Series object

In [45]:
def pick_most_frequent(values):
    if len(values.mode()) > 0:
        return values.mode()[0]
    else:
        return pd.NA

pick_most_frequent(pd.Series(['A', 'B', 'B', 'C']))

'B'

Pick the most frequent ML method for each department.

In [46]:
most_frequent_ml_method = df.groupby("Department")["ML Method"].agg(pd.Series.mode)
print(most_frequent_ml_method)

Department
Customer Support     Linear Regression
Finance                            RNN
Human Resources     [RNN, Transformer]
Marketing                Decision Tree
Operations                 Transformer
R&D                                RNN
Name: ML Method, dtype: object


Calculate the CO2 emissions for each department in different Green Energy categories. That is, the resulting dataframe will have as many colums as there are values for Green Energy.

In [47]:
category_co2 = df.groupby(['Department', 'Green Energy'])['co2_emissions_kg'].sum().unstack(fill_value=0)

# Ensure all expected categories are present, even if they're not in the data
for category in ['Green', 'Hybrid', 'Unknown']:
    if category not in category_co2:
        category_co2[category] = 0

category_co2.head()

Green Energy,Green,Hybrid,Unknown
Department,,,
Customer Support,1463.697220,1425.139532,9676.733146
Finance,2991.539843,2427.085307,8150.012032
Human Resources,2423.439560,3431.874604,9400.921879
Marketing,2282.675323,2923.163567,7615.917236
Operations,2253.368372,3779.599277,8971.934059


Combine everything into a final DataFrame

In [48]:
ml_method_mode = df.groupby('Department')['ML Method'].apply(pick_most_frequent).rename('ML Method')

# Combine everything into a final DataFrame
df_final = pd.DataFrame(total_co2).join([ml_method_mode, category_co2]).reset_index()

df_final.head()

,Department,co2_emissions_kg,ML Method,Green,Hybrid,Unknown
0,Customer Support,12565.569898,Linear Regression,1463.697220,1425.139532,9676.733146
1,Finance,13568.637182,RNN,2991.539843,2427.085307,8150.012032
2,Human Resources,15256.236043,RNN,2423.439560,3431.874604,9400.921879
3,Marketing,12821.756125,Decision Tree,2282.675323,2923.163567,7615.917236
4,Operations,15004.901708,Transformer,2253.368372,3779.599277,8971.934059


Make sure that the rows are sorted according to CO2 emissions in a way that the department with the largest emissions is first.

In [49]:
df_final = df_final.sort_values(by='co2_emissions_kg', ascending=False)
df_final.head()

,Department,co2_emissions_kg,ML Method,Green,Hybrid,Unknown
2,Human Resources,15256.236043,RNN,2423.439560,3431.874604,9400.921879
4,Operations,15004.901708,Transformer,2253.368372,3779.599277,8971.934059
5,R&D,14644.874294,RNN,3028.493500,2280.103568,9336.277226
1,Finance,13568.637182,RNN,2991.539843,2427.085307,8150.012032
3,Marketing,12821.756125,Decision Tree,2282.675323,2923.163567,7615.917236


In [50]:
import os

def ensure_folder_exists(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder '{folder_path}' created.")
    else:
        print(f"Folder '{folder_path}' already exists.")

ensure_folder_exists('results')

Folder 'results' already exists.


Finally, save the results.

In [51]:
df_final.to_excel('results/department_co2.xlsx', index=False)
df_final.to_pickle('results/department_co2.pkl')

PermissionError: [Errno 13] Permission denied: 'results/department_co2.xlsx'